In [4]:
import numpy as np
import pandas as pd
from hyperopt import hp
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

from hyperopt import STATUS_OK
from hyperopt import fmin,tpe,Trials

from sklearn.model_selection import cross_val_score

from xgboost import XGBClassifier

import warnings
warnings.filterwarnings('ignore')

dataset=load_breast_cancer()
X_features=dataset.data
y_label=dataset.target
cancer_dr=pd.DataFrame(
    data=X_features
    ,columns=dataset.feature_names
)
cancer_dr['target']=y_label

X_features=cancer_dr.iloc[:,:-1] 
y_label=cancer_dr.iloc[:,-1] 

X_train,X_test,y_train,y_test=train_test_split(
    X_features
    ,y_label
    ,train_size=0.8
    ,random_state=156
)

X_tr,X_val,y_tr,y_val=train_test_split(
    X_train
    ,y_train
    ,test_size=0.1
    ,random_state=156
)

# XGBoost 모델에 대한 하이퍼파라미터 검색 공간을 정의
xgb_search_space = {
    'max_depth': hp.quniform('max_depth', 5, 20, 1),  # 트리 깊이: 5에서 20까지 1씩 증가하는 정수 값
    'min_child_weight': hp.quniform('min_child_weight', 1, 2, 1),  # 최소 자식 노드 가중치: 1에서 2까지 1씩 증가하는 정수 값
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),  # 학습률: 0.01에서 0.2 사이의 연속값
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1)  # 각 트리의 컬럼 샘플 비율: 0.5에서 1 사이의 연속값
}

# 목적 함수 정의: 주어진 하이퍼파라미터 값을 받아 XGBoost 모델의 성능을 평가
def objective_func(search_space):
    # XGBoost 분류기 객체 생성
    xgb_clf = XGBClassifier(
        n_estimators=100,  # 100개의 트리를 사용
        max_depth=int(search_space['max_depth']),  # max_depth를 정수로 변환하여 설정
        min_child_weight=int(search_space['min_child_weight']),  # min_child_weight를 정수로 변환하여 설정
        learning_rate=search_space['learning_rate'],  # learning_rate는 연속값으로 설정
        colsample_bytree=search_space['colsample_bytree'],  # colsample_bytree는 연속값으로 설정
        eval_metric='logloss'  # 평가 지표로 로그 손실 사용
    )
    
    # 교차 검증을 통해 모델 정확도 평가
    accuracy = cross_val_score(
        xgb_clf # 점수를 구할 모델 설정
        , X_train # 학습데이터 전체를 넣어야한다.
        , y_train # 학습데이터의 값을 넣어야한다.
        , scoring='accuracy' # 평가 점수 : 정화고들 평가
        , cv=3  # 3겹 교차 검증 사용 : 정확도가 3개가 나온다
    ) 
    
    # 반환값: 평균 정확도를 음수로 변환하여 최소화하려는 함수로 반환
    # -1 * np.mean(accuracy)에서 -1을 곱하는 이유는 최적화 함수의 목적에 맞게 손실(loss) 값을 최소화하는 방향으로 Hyperopt를 사용할 수 있도록 하기 위해서입니다.
    # 정확도(accuracy)는 높을수록 좋습니다. 하지만 Hyperopt는 최소화를 목표로 하므로, 정확도가 높을수록 손실 값(loss)은 낮아져야 하므로, 음수 값으로 변환하여야 합니다.
    # cross_val_score로 얻은 accuracy는 최대화하려는 값입니다.
    # Hyperopt는 최소화 문제를 풀기 때문에, accuracy 값을 음수로 변환하여 최소화하려는 목적에 맞게 사용합니다.
    return {'loss': -1 * np.mean(accuracy), 'status': STATUS_OK}

# 하이퍼파라미터 튜닝 결과를 저장할 Trials 객체 생성
trial_val = Trials()

# Hyperopt의 fmin 함수 사용하여 하이퍼파라미터 튜닝 시작
best = fmin(
    fn=objective_func  # 최적화할 함수
    ,space=xgb_search_space  # 하이퍼파라미터 검색 공간
    ,algo=tpe.suggest  # 트리 구조를 사용하는 베이지안 최적화 방법
    ,max_evals=50  # 최대 평가 횟수 50번
    ,trials=trial_val  # 튜닝 진행 상황을 기록할 Trials 객체
    ,rstate=np.random.default_rng(seed=9)  # 난수 생성기의 시드 설정
)

# 최적화 결과 출력
print(f'best : {best}')

output = 'colsample_bytree:{0}, learning_rate:{1}, max_depth:{2}, min_child_weight:{3}'.format(
    round(best['colsample_bytree'], 5),
    round(best['learning_rate'], 5),
    int(best['max_depth']),
    int(best['min_child_weight'])
)

print(output)

100%|██████████| 50/50 [00:11<00:00,  4.24trial/s, best loss: -0.9692401533635412]
best : {'colsample_bytree': 0.7816916965882073, 'learning_rate': 0.15978354626451718, 'max_depth': 20.0, 'min_child_weight': 2.0}
colsample_bytree:0.78169, learning_rate:0.15978, max_depth:20, min_child_weight:2


In [5]:
import numpy as np
import pandas as pd
from hyperopt import hp
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

from hyperopt import STATUS_OK
from hyperopt import fmin,tpe,Trials

from sklearn.model_selection import cross_val_score

from xgboost import XGBClassifier

import warnings
warnings.filterwarnings('ignore')

dataset=load_breast_cancer()
X_features=dataset.data
y_label=dataset.target
cancer_dr=pd.DataFrame(
    data=X_features
    ,columns=dataset.feature_names
)
cancer_dr['target']=y_label

X_features=cancer_dr.iloc[:,:-1] 
y_label=cancer_dr.iloc[:,-1] 

X_train,X_test,y_train,y_test=train_test_split(
    X_features
    ,y_label
    ,train_size=0.8
    ,random_state=156
)

X_tr,X_val,y_tr,y_val=train_test_split(
    X_train
    ,y_train
    ,test_size=0.1
    ,random_state=156
)

xgb_search_space={
    'max_depth':hp.quniform('max_depth',5,20,1)
    ,'min_child_weight':hp.quniform('min_child_weight',1,2,1)
    ,'learning_rate':hp.uniform('learning_rate',0.01,0.2)
    ,'colsample_bytree':hp.uniform('colsample_bytree',0.5,1)
}

def objective_func(search_space):
    xgb_clf = XGBClassifier(
        n_estimators=100,
        max_depth=int(search_space['max_depth']),
        min_child_weight=int(search_space['min_child_weight']),
        learning_rate=search_space['learning_rate'],
        colsample_bytree=search_space['colsample_bytree'],
        eval_metric='logloss',
        tree_method='gpu_hist',
        gpu_id=0
        ,verbosity=2  # 디버그 모드를 활성화
        ,max_delta_step=5
    )

    accuracy=cross_val_score(xgb_clf,X_train,y_train,scoring='accuracy',cv=3)
    
    return {'loss':-1*np.mean(accuracy), 'status' : STATUS_OK}

trial_val = Trials()

best_gpu=fmin(
    fn=objective_func
    ,space=xgb_search_space
    ,algo=tpe.suggest
    ,max_evals=50
    ,trials=trial_val
    ,rstate=np.random.default_rng(seed=9)
    ,verbose=True
)

print(f'best : {best_gpu}')

output = 'colsample_bytree:{0}, learning_rate:{1}, max_depth:{2}, min_child_weight:{3}'.format(
    round(best['colsample_bytree'], 5),
    round(best['learning_rate'], 5),
    int(best['max_depth']),
    int(best['min_child_weight'])
)

print(output)

[11:53:36] INFO: C:\actions-runner\_work\xgboost\xgboost\src\data\iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (303, 30, 9090).
[11:53:36] INFO: C:\actions-runner\_work\xgboost\xgboost\src\data\ellpack_page.cu:167: Ellpack is dense.
[11:53:37] INFO: C:\actions-runner\_work\xgboost\xgboost\src\data\iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (303, 30, 9090).
[11:53:37] INFO: C:\actions-runner\_work\xgboost\xgboost\src\data\ellpack_page.cu:167: Ellpack is dense.
[11:53:37] INFO: C:\actions-runner\_work\xgboost\xgboost\src\data\iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (304, 30, 9120).
[11:53:37] INFO: C:\actions-runner\_work\xgboost\xgboost\src\data\ellpack_page.cu:167: Ellpack is dense.
[11:53:37] INFO: C:\actions-runner\_work\xgboost\xgboost\src\data\iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (303, 30, 9090).
[11:53:37] INFO: C:\actions-runner\_work\xgboost\xgboost\src\data\ell

In [6]:
import numpy as np
import pandas as pd
from hyperopt import hp
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

from hyperopt import STATUS_OK
from hyperopt import fmin,tpe,Trials

from sklearn.model_selection import cross_val_score

from xgboost import XGBClassifier

import warnings
warnings.filterwarnings('ignore')

dataset=load_breast_cancer()
X_features=dataset.data
y_label=dataset.target
cancer_dr=pd.DataFrame(
    data=X_features
    ,columns=dataset.feature_names
)
cancer_dr['target']=y_label

X_features=cancer_dr.iloc[:,:-1] 
y_label=cancer_dr.iloc[:,-1] 

X_train,X_test,y_train,y_test=train_test_split(
    X_features
    ,y_label
    ,train_size=0.8
    ,random_state=156
)

X_tr,X_val,y_tr,y_val=train_test_split(
    X_train
    ,y_train
    ,test_size=0.1
    ,random_state=156
)

# XGBoost 모델에 대한 하이퍼파라미터 검색 공간을 정의
xgb_search_space = {
    'max_depth': hp.quniform('max_depth', 5, 20, 1),  # 트리 깊이: 5에서 20까지 1씩 증가하는 정수 값
    'min_child_weight': hp.quniform('min_child_weight', 1, 2, 1),  # 최소 자식 노드 가중치: 1에서 2까지 1씩 증가하는 정수 값
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),  # 학습률: 0.01에서 0.2 사이의 연속값
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1)  # 각 트리의 컬럼 샘플 비율: 0.5에서 1 사이의 연속값
}

# 목적 함수 정의: 주어진 하이퍼파라미터 값을 받아 XGBoost 모델의 성능을 평가
def objective_func(search_space):
    # XGBoost 분류기 객체 생성
    xgb_clf = XGBClassifier(
        n_estimators=100,  # 100개의 트리를 사용
        max_depth=int(search_space['max_depth']),  # max_depth를 정수로 변환하여 설정
        min_child_weight=int(search_space['min_child_weight']),  # min_child_weight를 정수로 변환하여 설정
        learning_rate=search_space['learning_rate'],  # learning_rate는 연속값으로 설정
        colsample_bytree=search_space['colsample_bytree'],  # colsample_bytree는 연속값으로 설정
        eval_metric='logloss'  # 평가 지표로 로그 손실 사용
    )
    
    # 교차 검증을 통해 모델 정확도 평가
    accuracy = cross_val_score(
        xgb_clf # 점수를 구할 모델 설정
        , X_train # 학습데이터 전체를 넣어야한다.
        , y_train # 학습데이터의 값을 넣어야한다.
        , scoring='accuracy' # 평가 점수 : 정화고들 평가
        , cv=3  # 3겹 교차 검증 사용 : 정확도가 3개가 나온다
    ) 
    
    # 반환값: 평균 정확도를 음수로 변환하여 최소화하려는 함수로 반환
    # -1 * np.mean(accuracy)에서 -1을 곱하는 이유는 최적화 함수의 목적에 맞게 손실(loss) 값을 최소화하는 방향으로 Hyperopt를 사용할 수 있도록 하기 위해서입니다.
    # 정확도(accuracy)는 높을수록 좋습니다. 하지만 Hyperopt는 최소화를 목표로 하므로, 정확도가 높을수록 손실 값(loss)은 낮아져야 하므로, 음수 값으로 변환하여야 합니다.
    # cross_val_score로 얻은 accuracy는 최대화하려는 값입니다.
    # Hyperopt는 최소화 문제를 풀기 때문에, accuracy 값을 음수로 변환하여 최소화하려는 목적에 맞게 사용합니다.
    return {'loss': -1 * np.mean(accuracy), 'status': STATUS_OK}

# 하이퍼파라미터 튜닝 결과를 저장할 Trials 객체 생성
trial_val = Trials()

# Hyperopt의 fmin 함수 사용하여 하이퍼파라미터 튜닝 시작
best = fmin(
    fn=objective_func  # 최적화할 함수
    ,space=xgb_search_space  # 하이퍼파라미터 검색 공간
    ,algo=tpe.suggest  # 트리 구조를 사용하는 베이지안 최적화 방법
    ,max_evals=50  # 최대 평가 횟수 50번
    ,trials=trial_val  # 튜닝 진행 상황을 기록할 Trials 객체
    ,rstate=np.random.default_rng(seed=9)  # 난수 생성기의 시드 설정
)

# 최적화 결과 출력
print(f'best : {best}')

output = 'colsample_bytree:{0}, learning_rate:{1}, max_depth:{2}, min_child_weight:{3}'.format(
    round(best['colsample_bytree'], 5),
    round(best['learning_rate'], 5),
    int(best['max_depth']),
    int(best['min_child_weight'])
)

print(output)

# 모델 생성 성능 평가 : get_clf_eval(원래답, 예측값, 예측확률)
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score


def get_clf_eval(y_test, pred=None, pred_proba=None):
    confusion = confusion_matrix( y_test, pred)
    accuracy = accuracy_score(y_test , pred) # 정확도 점수
    precision = precision_score(y_test , pred) # 정밀도 점수
    recall = recall_score(y_test , pred) # 재현율 점수
    f1 = f1_score(y_test,pred) # 정밀도, 재현율 조화평균 값
    # ROC-AUC 추가
    roc_auc = roc_auc_score(y_test, pred_proba) # AUC 점수 : 불균형 데이터 셋에서 필요
    print('오차 행렬')
    print(confusion)
    # ROC-AUC print 추가
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f},\
    F1: {3:.4f}, AUC:{4:.4f}'.format(accuracy, precision, recall, f1, roc_auc))

xgb_wrapper = XGBClassifier(n_estimators=1000,
                            learning_rate=round(best['learning_rate'], 5),
                            max_depth=int(best['max_depth']),
                            min_child_weight=int(best['min_child_weight']),
                            colsample_bytree=round(best['colsample_bytree'], 5)
                            , early_stopping_rounds=50
                            , eval_metric='logloss'
                           )


evals = [(X_tr, y_tr), (X_val, y_val)]
xgb_wrapper.fit(
    X_tr
    , y_tr
    ,eval_set=evals
    , verbose=True
)

preds = xgb_wrapper.predict(X_test) # 예측값
pred_proba = xgb_wrapper.predict_proba(X_test)[:, 1] # 예측 확률

get_clf_eval(y_test, preds, pred_proba)


100%|██████████| 50/50 [00:12<00:00,  3.89trial/s, best loss: -0.9692401533635412]
best : {'colsample_bytree': 0.7816916965882073, 'learning_rate': 0.15978354626451718, 'max_depth': 20.0, 'min_child_weight': 2.0}
colsample_bytree:0.78169, learning_rate:0.15978, max_depth:20, min_child_weight:2
[0]	validation_0-logloss:0.54379	validation_1-logloss:0.58107
[1]	validation_0-logloss:0.45190	validation_1-logloss:0.51622
[2]	validation_0-logloss:0.38041	validation_1-logloss:0.45865
[3]	validation_0-logloss:0.32496	validation_1-logloss:0.41740
[4]	validation_0-logloss:0.28110	validation_1-logloss:0.38619
[5]	validation_0-logloss:0.24305	validation_1-logloss:0.35807
[6]	validation_0-logloss:0.21292	validation_1-logloss:0.34329
[7]	validation_0-logloss:0.18874	validation_1-logloss:0.32475
[8]	validation_0-logloss:0.16768	validation_1-logloss:0.31701
[9]	validation_0-logloss:0.14904	validation_1-logloss:0.30362
[10]	validation_0-logloss:0.13466	validation_1-logloss:0.29923
[11]	validation_0-logl